# ANN to model Cl

In [18]:
# Create dataset
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset

results_dir = "results/"


class AerofoilLiftSample:
    def __init__(self, file_name):
        self.file_name = file_name
        aerofoil_data = self.file_name.split("_")
        naca = aerofoil_data[1]
        self.camber = naca[0]
        self.camber_pos = naca[1]
        self.thickness = naca[2:3]
        self.reynolds = aerofoil_data[2]
        self.mach = aerofoil_data[3]
        self.alpha = aerofoil_data[4].split(".")[0]

    def __str__(self):
        return self.file_name

    def cp_data(self):
        file = results_dir + self.file_name
        df = pd.read_csv(file, sep="\s+", skiprows=[0,1,2], names=["x", "y", "Cp"], dtype=float)
        return df

    def cl(self):
        return np.trapz(self.cp_data()["Cp"], self.cp_data()["x"])

class AerofoilLiftDataset(Dataset):
    """Aerofoil Coefficient of Lift Dataset"""
    def __init__(self):
        self.samples = self.load_samples()

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, item):
        return self.samples[item]

    def load_samples(self):
        samples = []
        for file in os.listdir(results_dir):
            samples.append(AerofoilLiftSample(file))
        return samples

In [19]:
dataset = AerofoilLiftDataset()
print(dataset[0].cl())

-0.5966156840500002
